# HT 6. Modelos de Regresión Logística

## Miembros
- Francisco Javier Castillo Cerna 21562
- Fernanda Esquivel de León 21542
- Andrés Estuardo Montoya 21552

## Link al repositorio 
El repositorio puede ser visualizado [acá](https://github.com/FerEsq/DM-House-Saleprice-Predictor)

## Link al documento
El documento puede ser visualizado [acá](https://docs.google.com/document/d/1epV70f3ZMXSBei_e2BAuRYk5oIA3SR9INmNR5e3FXoE/edit?usp=sharing)

# Regresión Logística para predecir si una vivienda es de cierto rango económico o no.

## 0. Preparación de datos

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, FunctionTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [42]:
df = pd.read_csv('data/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [43]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [44]:
df["MSZoning"] = le.fit_transform(df["MSZoning"])
df["Street"] = le.fit_transform(df["Street"])
df["Alley"] = le.fit_transform(df["Alley"])
df["LotShape"] = le.fit_transform(df["LotShape"])
df["LandContour"] = le.fit_transform(df["LandContour"])
df["Utilities"] = le.fit_transform(df["Utilities"])
df["LotConfig"] = le.fit_transform(df["LotConfig"])
df["LandSlope"] = df["LandSlope"].map({
    "Gtl": 1,
    "Mod": 2,
    "Sev": 3
})
df["Neighborhood"] = le.fit_transform(df["Neighborhood"])
df["Condition1"] = le.fit_transform(df["Condition1"])
df["Condition2"] = le.fit_transform(df["Condition2"])
df["BldgType"] = le.fit_transform(df["BldgType"])
df["HouseStyle"] = le.fit_transform(df["HouseStyle"])
df["RoofStyle"] = le.fit_transform(df["RoofStyle"])
df["RoofMatl"] = le.fit_transform(df["RoofMatl"])
df["Exterior1st"] = le.fit_transform(df["Exterior1st"])
df["Exterior2nd"] = le.fit_transform(df["Exterior2nd"])
df["MasVnrType"] = le.fit_transform(df["MasVnrType"])

condition_map = {
    "Ex": 5,
    "Gd": 4,
    "TA": 3,
    "Fa": 2,
    "Po": 1,
    "NA": 0,
}
df["ExterQual"] = df["ExterQual"].map(condition_map)
df["ExterCond"] = df["ExterCond"].map(condition_map)
df["Foundation"] = le.fit_transform(df["Foundation"])
df["BsmtQual"] = df["BsmtQual"].map(condition_map)
df["BsmtCond"] = df["BsmtCond"].map(condition_map)
df["BsmtExposure"] = df["BsmtExposure"].map({
    "Gd": 4,
    "Av": 3,
    "Mn": 2,
    "No": 1,
    "NA": 0
})
df["BsmtFinType1"] = df["BsmtFinType1"].map({
    "GLQ": 6,
    "ALQ": 5,
    "BLQ": 4,
    "Rec": 3,
    "LwQ": 2,
    "Unf": 1,
    "NA": 0
})
df["BsmtFinType2"] = df["BsmtFinType2"].map({
    "GLQ": 6,
    "ALQ": 5,
    "BLQ": 4,
    "Rec": 3,
    "LwQ": 2,
    "Unf": 1,
    "NA": 0
})
df["Heating"] = le.fit_transform(df["Heating"])
df["HeatingQC"] = df["HeatingQC"].map(condition_map)
df["CentralAir"] = le.fit_transform(df["CentralAir"])
df["Electrical"] = df["Electrical"].map({
    "SBrkr": 5,
    "FuseA": 4,
    "FuseF": 3,
    "FuseP": 2,
    "Mix": 1,
    "NA": 0
})
df["KitchenQual"] = df["KitchenQual"].map(condition_map)
df["Functional"] = le.fit_transform(df["Functional"])
df["FireplaceQu"] = df["FireplaceQu"].map(condition_map)
df["GarageType"] = le.fit_transform(df["GarageType"])
df["GarageFinish"] = df["GarageFinish"].map({
    "Fin": 3,
    "RFn": 2,
    "Unf": 1,
    "NA": 0
})
df["GarageQual"] = df["GarageQual"].map(condition_map)
df["GarageCond"] = df["GarageCond"].map(condition_map)
df["PavedDrive"] = df["PavedDrive"].map({
    "Y": 3,
    "P": 2,
    "N": 1,
    "NA": 0
})
df["PoolQC"] = df["PoolQC"].map(condition_map)
df["Fence"] = df["Fence"].map({
    "GdPrv": 4,
    "MnPrv": 3,
    "GdWo": 2,
    "MnWw": 1,
    "NA": 0
})
df["MiscFeature"] = le.fit_transform(df["MiscFeature"])
df["SaleType"] = le.fit_transform(df["SaleType"])
df["SaleCondition"] = le.fit_transform(df["SaleCondition"])

for col in df.columns:
    if df[col].isna().sum() > 0:
        df[col] = df[col].fillna(0)

## 1. Cree una variable dicotómica...

In [45]:
price_class_df = df.copy()
lower_limit = np.percentile(price_class_df['SalePrice'], 33)
upper_limit = np.percentile(price_class_df['SalePrice'], 66)


def classify_house(price):
    if price < lower_limit:
        return 'Economical'
    elif price < upper_limit:
        return 'Intermediate'
    else:
        return 'Expensive'


price_class_df['Classification'] = price_class_df['SalePrice'].apply(classify_house)

print(price_class_df[['SalePrice', 'Classification']].head())

   SalePrice Classification
0     208500      Expensive
1     181500   Intermediate
2     223500      Expensive
3     140000   Intermediate
4     250000      Expensive


In [46]:
price_class_df['IsExpensive'] = (price_class_df['Classification'] == 'Expensive').astype(int)
price_class_df['IsIntermediate'] = (price_class_df['Classification'] == 'Intermediate').astype(int)
price_class_df['IsEconomical'] = (price_class_df['Classification'] == 'Economical').astype(int)

In [47]:
print(price_class_df[['Classification', 'IsExpensive', 'IsIntermediate', 'IsEconomical']].head())

  Classification  IsExpensive  IsIntermediate  IsEconomical
0      Expensive            1               0             0
1   Intermediate            0               1             0
2      Expensive            1               0             0
3   Intermediate            0               1             0
4      Expensive            1               0             0


## 2. Utilice los mismos conjuntos de entrenamiento y prueba...

In [48]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np

In [49]:
X = price_class_df.drop(columns=["Id", "SalePrice", "Classification", "IsExpensive", "IsIntermediate", "IsEconomical"])
y_expensive = price_class_df["IsExpensive"]
y_intermediate = price_class_df["IsIntermediate"]
y_economical = price_class_df["IsEconomical"]

In [50]:
test_size = 0.2
random_state = 42

X_train_expensive, X_test_expensive, y_train_expensive, y_test_expensive = train_test_split(X, y_expensive,
                                                                                            test_size=test_size,
                                                                                            random_state=random_state)
X_train_intermediate, X_test_intermediate, y_train_intermediate, y_test_intermediate = train_test_split(X,
                                                                                                        y_intermediate,
                                                                                                        test_size=test_size,
                                                                                                        random_state=random_state)
X_train_economical, X_test_economical, y_train_economical, y_test_economical = train_test_split(X, y_economical,
                                                                                                test_size=test_size,
                                                                                                random_state=random_state)

## 3. Elabore un modelo de regresión logística...

In [51]:
solver = 'liblinear'
max_iters = 1_500

model_economical = LogisticRegression(solver=solver, max_iter=max_iters)
model_intermediate = LogisticRegression(solver=solver, max_iter=max_iters)
model_expensive = LogisticRegression(solver=solver, max_iter=max_iters)

In [52]:
model_economical = model_economical.fit(X_train_economical, y_train_economical)
model_intermediate = model_intermediate.fit(X_train_intermediate, y_train_intermediate)
model_expensive = model_expensive.fit(X_train_expensive, y_train_expensive)

### 3.1. Utilice validación cruzada

In [53]:
# Fit the model to your data
model_economical.fit(X_train_economical, y_train_economical)
model_intermediate.fit(X_train_intermediate, y_train_intermediate)
model_expensive.fit(X_train_expensive, y_train_expensive)

LogisticRegression(max_iter=1500, solver='liblinear')

In [54]:
# Perform cross-validation
scores = {
    'economical': cross_val_score(model_economical, X, y_economical, cv=5),
    'intermediate': cross_val_score(model_intermediate, X, y_intermediate, cv=5),
    'expensive': cross_val_score(model_expensive, X, y_expensive, cv=5)
}

In [55]:
for key, value in scores.items():
    print(f'Cross validation score of {key} model: {value.mean()}')

Cross validation score of economical model: 0.9041095890410957
Cross validation score of intermediate model: 0.7054794520547946
Cross validation score of expensive model: 0.9191780821917808


### 3.2. Resultados obtenidos

Podemos observar que los valores obtenidos con un entrenamiento de validación cruzada han sido altos; pero no lo suficientemente para poder afirmar que el modelo sufre de overfitting. En conclusión, podemos ver que el modelo de regresión logística nos permite hacer clasificaciones con un alto nivel de precisión.

## 4. Analice el modelo

In [56]:
# # Cálculo de VIF para cada variable
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.tools.tools import add_constant
# import statsmodels.api as sm
# 
# X_train_const = add_constant(X_train)
# 
# vif_data = pd.DataFrame()
# vif_data['feature'] = X_train_const.columns
# vif_data['VIF'] = [variance_inflation_factor(X_train_const.values, i) for i in range(X_train_const.shape[1])]
# 
# print(vif_data)


D:\UVG\DataMining\.venv\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


          feature           VIF
0           const  2.599763e+06
1      MSSubClass  5.533222e+00
2        MSZoning  1.400416e+00
3     LotFrontage  1.402632e+00
4         LotArea  1.776529e+00
..            ...           ...
75        MiscVal  1.693587e+00
76         MoSold  1.096128e+00
77         YrSold  1.103613e+00
78       SaleType  1.164254e+00
79  SaleCondition  1.247673e+00

[80 rows x 2 columns]


In [57]:
# #Ajuste del modelo de regresión logística y examinación de los p-values de las variables
# logit_model = sm.Logit(y_train, X_train_const)
# logit_result = logit_model.fit()
# 
# print(logit_result.summary())


         Current function value: 0.175716
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:            IsExpensive   No. Observations:                 1168
Model:                          Logit   Df Residuals:                     1088
Method:                           MLE   Df Model:                           79
Date:                Thu, 11 Apr 2024   Pseudo R-squ.:                  0.7271
Time:                        16:01:23   Log-Likelihood:                -205.24
converged:                      False   LL-Null:                       -751.92
Covariance Type:            nonrobust   LLR p-value:                3.161e-178
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const          -278.1675    219.475     -1.267      0.205    -708.330     151.995
MSSubClass       -0.0269      0.014     -1.882      0.06

D:\UVG\DataMining\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [58]:
# #Evaluación el ajuste del modelo
# X_test_const = add_constant(X_test)
# 
# #Se usa el resultado del modelo ajustado para hacer predicciones
# y_pred = logit_result.predict(X_test_const)
# 
# #Se calcula la precisión y el AUC con las predicciones
# accuracy = accuracy_score(y_test, y_pred.round())
# auc = roc_auc_score(y_test, y_pred)
# 
# print(f'Precisión: {accuracy}')
# print(f'AUC: {auc}')

NameError: name 'roc_auc_score' is not defined


### 4.1. Análisis de los resultados
Los resultados muestran que el modelo de regresión logística tiene un pseudo R-cuadrado de 0.7271, lo que indica que el modelo explica aproximadamente el 72.71% de la variabilidad en la clasificación de las viviendas como caras o no. Este valor sugiere que el modelo tiene un buen ajuste. 

En cuanto a la multicolinealidad, el VIF para la mayoría de las variables está por debajo de 10, lo que sugiere que no hay problemas graves de multicolinealidad que afecten al modelo.

Observando los p-values en el resumen del modelo, varias variables, como MSZoning, Alley, y LotShape, tienen p-values menores de 0.05, indicando que son significativas en el modelo. Esto significa que estas variables contribuyen de manera significativa a la predicción de si una vivienda es cara o no.

En términos de rendimiento del modelo, la precisión es del 92.12% y el AUC es de 0.980, ambos indicadores muy altos, lo que demuestra que el modelo es muy efectivo en clasificar correctamente las viviendas.

En conclusión, el modelo parece adaptarse bien a los datos, con un buen nivel de explicación de la variabilidad y un rendimiento excelente en la clasificación. .

## 5. Clasifique el conjunto de prueba

In [ ]:
# from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
# 
# X_test_const = add_constant(X_test)
# 
# #Se usa el resultado del modelo ajustado para hacer predicciones
# y_pred = logit_result.predict(X_test_const)
# 
# #Calcular la precisión, AUC y otras métricas
# accuracy = accuracy_score(y_test, y_pred.round())
# auc = roc_auc_score(y_test, y_pred)
# conf_matrix = confusion_matrix(y_test, y_pred.round())
# class_report = classification_report(y_test, y_pred.round())
# 
# print(f'Precisión: {accuracy}')
# print(f'AUC: {auc}')
# print('Matriz de confusión:')
# print(conf_matrix)
# print('Reporte de clasificación:')
# print(class_report)



La eficiencia del modelo de regresión puede considerar alta, pues cuenta con una precisión del 92.12% y un AUC de 0.980. Estas métricas indican un excelente rendimiento del modelo en la diferenciación entre las clases. La matriz de confusión muestra que el modelo ha clasificado correctamente 185 viviendas como no caras y 84 como caras, con solo 12 y 11 errores de clasificación, respectivamente, lo que refleja una buena capacidad de generalización.

Además, el reporte de clasificación detalla una precisión y recall de 0.94 y 0.88 para las clases no cara y cara respectivamente, lo cual sugiere que el modelo es equilibrado en términos de sensibilidad y especificidad. 

## 6. ¿Hay overfitting? Muestre curvas de aprendizaje

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.model_selection import learning_curve
# 
# train_sizes, train_scores, test_scores = learning_curve(
#     LogisticRegression(solver='liblinear'),
#     X, y,
#     cv=5,
#     scoring='neg_mean_squared_error',
#     n_jobs=-1,
#     train_sizes=np.linspace(.1, 1.0, 5)
# )
# 
# #Calculo de las medias y desviaciones estándar del error de entrenamiento y prueba
# train_scores_mean = -np.mean(train_scores, axis=1)
# train_scores_std = np.std(train_scores, axis=1)
# test_scores_mean = -np.mean(test_scores, axis=1)
# test_scores_std = np.std(test_scores, axis=1)
# 
# #Se grafican las curvas de aprendizaje
# plt.figure()
# plt.title("Curvas de aprendizaje")
# plt.xlabel("Tamaño del conjunto de entrenamiento")
# plt.ylabel("Error")
# plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
#                  train_scores_mean + train_scores_std, alpha=0.1,
#                  color="r")
# plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
#                  test_scores_mean + test_scores_std, alpha=0.1, color="g")
# plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
#          label="Error de entrenamiento")
# plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
#          label="Error de prueba")
# 
# plt.legend(loc="best")
# plt.show()



La curva de aprendizaje nos muestra que no hay sobreajuste significativo en el modelo de regresión logística. El error de entrenamiento muestra un aumento marginal con el incremento en el tamaño del conjunto de entrenamiento, mientras que el error de prueba disminuye, y ambas curvas tienden a converger. La proximidad de las dos curvas y la disminución de su desviación estándar indican que el modelo mantiene una buena generalización a medida que se introducen más datos. Esto sugiere que el modelo está bien ajustado, con un rendimiento estable en los datos de entrenamiento y de prueba, lo cual es indicativo de una buena capacidad de generalización y una eficiencia adecuada en la clasificación de viviendas como caras o no.

## 7. Tuneo de hiperparámetros

In [59]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Define the hyperparameters and their values
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(model_economical, param_grid, cv=5)

# Fit the GridSearchCV object to your data
grid_search.fit(X_train_economical, y_train_economical)

# Print the best parameters
print('Best parameters:', grid_search.best_params_)

Best parameters: {'C': 10, 'penalty': 'l2'}
